# akademisyenParsing
#### by Dr. Öğr. Üy. Mehmet Selim Yavuz
###### İstanbul Üniversitesi, Müzikoloji Departmanı
27/10/2022
<br>
Bu kod YÖK Akademik veritabanından dışarıya aktarılan akademisyen listelerini csv formatına dönüştürmek içindir.

Lazım olan kütüphaneleri dahil edelim.

In [32]:
import numpy as np
import pandas as pd
import os
import fileinput
import operator

Bütün listeleri okuyup içeri alalım.

In [112]:
workingDir = os.getcwd()
targetDir = "/imports/"
path = workingDir + targetDir

dirList = os.listdir(path)
#print(dirList)

akademisyenList = []
for txtFile in dirList:
    #print(txtFile)
    if txtFile.endswith('.txt'):
        with open(path + txtFile) as f:
            for line in f:
                if line.isspace() != True: #print("!!!!BURADA SIPEYS VAR!!!!")
                    akademisyenList.append(line.rstrip())
    #else:
    #    print("!!!! " + txtFile + " TXT DOSYASI DEĞİL!!!!")

print(akademisyenList[0])

103448, ÖĞRETİM GÖREVLİSİ, KAZIM GÜRKAN, SAKARYA ÜNİVERSİTESİ/DEVLET KONSERVATUVARI/ÇALGI EĞİTİMİ BÖLÜMÜ/TÜRK HALK MÜZİĞİ ANABİLİM DALI/, Güzel Sanatlar Temel Alanı, Müzik, Türk Halk müziği ; Türk Halk Müziği (Yorumculuk), kgurkan[at]sakarya.edu.tr, BD15416E4C4254DB


Artık `akademisyenList`de YÖK formatında bilgiler mevcut.

YÖK formatı: akademisyenID, titr, isim-soyisim, üniversite/fakülte/departman/ABD/, temel_alan, bilim_alanı, anahtar_kelime (varsa) ; anahtar_kelime2 (varsa) ; anahtar_kelime3 (varsa), e-posta_adresi, key

Teker teker bakabilmek için liste objelerini listeye çevirmek gerekiyor.

In [113]:
akademisyen = 0
for index, akademisyen in enumerate(akademisyenList):
    akademisyenList[index] = akademisyen.split(", ") # YÖK formatında virgülle ayrılan alanları ayır.
    akademisyenList[index] = akademisyenList[index][:4] # Temel bilim alanı, bilim alanı ve anahtar kelimelere ihtiyacımız yok.
    #print(akademisyenList[index])

Nispeten temiz bir liste elimizde. Artık üniversite kısmını ayırmak gerekiyor.

In [114]:
for index, akademisyen in enumerate(akademisyenList):
    #print(akademisyen)
    splitList = akademisyen[3].split("/")
    #print(splitList)
    cleanList = akademisyen + splitList
    cleanList.pop(3) # eski listeye ihtiyacımız yok
    print(cleanList)
    cleanList.pop(4) # fakülte bilgisine ihtiyacımız yok
    print(cleanList)
    if cleanList[-1] == "": # sonda bazen boşluk var
        cleanList.pop(-1)
        #print(cleanList)
    if len(cleanList) == 6:
        cleanList.pop(4)
        print(cleanList)
    akademisyenList[index] = cleanList
    #print(akademisyen)

['103448', 'ÖĞRETİM GÖREVLİSİ', 'KAZIM GÜRKAN', 'SAKARYA ÜNİVERSİTESİ', 'DEVLET KONSERVATUVARI', 'ÇALGI EĞİTİMİ BÖLÜMÜ', 'TÜRK HALK MÜZİĞİ ANABİLİM DALI', '']
['103448', 'ÖĞRETİM GÖREVLİSİ', 'KAZIM GÜRKAN', 'SAKARYA ÜNİVERSİTESİ', 'ÇALGI EĞİTİMİ BÖLÜMÜ', 'TÜRK HALK MÜZİĞİ ANABİLİM DALI', '']
['103448', 'ÖĞRETİM GÖREVLİSİ', 'KAZIM GÜRKAN', 'SAKARYA ÜNİVERSİTESİ', 'TÜRK HALK MÜZİĞİ ANABİLİM DALI']
['117411', 'ÖĞRETİM GÖREVLİSİ', 'TUNCAY KARDAŞ', 'SAKARYA ÜNİVERSİTESİ', 'DEVLET KONSERVATUVARI', 'ÇALGI EĞİTİMİ BÖLÜMÜ', 'TÜRK SANAT MÜZİĞİ ANABİLİM DALI', '']
['117411', 'ÖĞRETİM GÖREVLİSİ', 'TUNCAY KARDAŞ', 'SAKARYA ÜNİVERSİTESİ', 'ÇALGI EĞİTİMİ BÖLÜMÜ', 'TÜRK SANAT MÜZİĞİ ANABİLİM DALI', '']
['117411', 'ÖĞRETİM GÖREVLİSİ', 'TUNCAY KARDAŞ', 'SAKARYA ÜNİVERSİTESİ', 'TÜRK SANAT MÜZİĞİ ANABİLİM DALI']
['103363', 'ÖĞRETİM GÖREVLİSİ', 'METİN GÜLSÜN', 'SAKARYA ÜNİVERSİTESİ', 'DEVLET KONSERVATUVARI', 'ÇALGI EĞİTİMİ BÖLÜMÜ', 'TÜRK SANAT MÜZİĞİ ANABİLİM DALI', '']
['103363', 'ÖĞRETİM GÖREVLİSİ', 'MET

Artık temiz bir akademisyen listemiz mevcut. Kalan format: [0] akademisyenID, [1] titr, [2] tam_isim, [3] üniversite, [4] abd

In [115]:
print(akademisyenList[0])

['103448', 'ÖĞRETİM GÖREVLİSİ', 'KAZIM GÜRKAN', 'SAKARYA ÜNİVERSİTESİ', 'TÜRK HALK MÜZİĞİ ANABİLİM DALI']
